In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import tensorflow as tf

In [75]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, GlobalAveragePooling1D

In [69]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences, to_categorical

In [9]:
df = pd.read_csv("./dialogs.txt", sep='\t', header=None, names=["pergunta", "resposta"])

In [10]:
df

,pergunta,resposta
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.
...,...,...
3720,that's a good question. maybe it's not old age.,are you right-handed?
3721,are you right-handed?,yes. all my life.
3722,yes. all my life.,you're wearing out your right hand. stop using...
3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [11]:
contraction_dict = {
    "aren't": "are not", "can't": "can not", "could've": "could have",
    "couldn't": "could not", "daren't": "dare not", "didn't": "did not",
    "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hasn't": "has not", "haven't": "have not", "he's": "he is",
    "how'd": "how had", "how're": "how are", "how's": "how is",
    "how've": "how have", "i'd": "i had", "i'm": "i am",
    "i've": "i have", "isn't": "is+ not", "it's": "it is",
    "might've": "might have", "mightn't": "might not", "must've": "must have",
    "mustn't": "must not", "needn't": "need not", "oughtn't": "ought not",
    "shan't": "shall not", "she'd": "she had", "she's": "she is",
    "should've": "should have", "shouldn't": "should not", "that'd": "that had",
    "thats's": "that is", "there'd": "there had", "there's": "there is",
    "they'd": "they had", "they're": "you are", "they've": "they have",
    "wasn't": "was+ not", "we'd": "we had", "we're": "we are",
    "we've": "we have", "weren't": "were not", "what'd": "what had",
    "what're": "what are", "what's": "what is", "what've": "what have",
    "when'd": "when had", "when're": "when are", "when's": "when is",
    "when've": "when have", "where'd": "where had", "where're": "where are",
    "where's": "where is", "where've": "where have", "who'd": "who had",
    "who're": "who are", "who's": "who is", "who've": "who have",
    "why'd": "why had", "why're": "why are", "why's": "why is",
    "why've": "why have", "would've": "would have", "wouldn't": "would not",
    "you're": "you are", "you've": "you have", "'cause": "because", 
    "ain't": "is not", "aren't": "are not", "can't": "cannot", 
    "could've": "could have", "he's": "he is", "how'll": "how will",
    "i'll": "i will", "it'll": "it will", "it's": "it is", 
    "she'll": "she will", "she's": "she is", "that'll": "that will",
    "there'll": "there will", "they'll": "they will", "they're": "they are",
    "we'll": "we will", "we're": "we are", "what'll": "what will",
    "when'll": "when will", "where'll": "where will", "who'll": "who will",
    "yo're": "you are", "you'll": "you will"
}

In [28]:
def limpar_texto( texto ):
    texto = texto.lower()
    texto = texto.replace('?', ' ?')
    texto = texto.replace('.', ' ')
    texto = texto.replace(',', ' ')
    texto = texto.replace('  ', ' ')
    tokens = texto.split(" ")
    lista = []
    for token in tokens:
        palavra = token
        if token in contraction_dict:
            palavra = contraction_dict[token]
        lista.append(palavra)
    return "<START> " + " ".join(lista).strip() + " <END>"

In [29]:
df["pergunta_limpa"] = df["pergunta"].apply(limpar_texto)
df["resposta_limpa"] = df["resposta"].apply(limpar_texto)

In [30]:
df

,pergunta,resposta,pergunta_limpa,resposta_limpa
0,"hi, how are you doing?",i'm fine. how about yourself?,<START> hi how are you doing ? <END>,<START> i am fine how about yourself ? <END>
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.,<START> i am fine how about yourself ? <END>,<START> i am pretty good thanks for asking <END>
2,i'm pretty good. thanks for asking.,no problem. so how have you been?,<START> i am pretty good thanks for asking <END>,<START> no problem so how have you been ? <END>
3,no problem. so how have you been?,i've been great. what about you?,<START> no problem so how have you been ? <END>,<START> i have been great what about you ? <END>
4,i've been great. what about you?,i've been good. i'm in school right now.,<START> i have been great what about you ? <END>,<START> i have been good i am in school right ...
...,...,...,...,...
3720,that's a good question. maybe it's not old age.,are you right-handed?,<START> that's a good question maybe it is not...,<START> are you right-handed ? <END>
3721,are you right-handed?,yes. all my life.,<START> are you right-handed ? <END>,<START> yes all my life <END>
3722,yes. all my life.,you're wearing out your right hand. stop using...,<START> yes all my life <END>,<START> you are wearing out your right hand st...
3723,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.,<START> you are wearing out your right hand st...,<START> but i do all my writing with my right ...


In [45]:
tokenizer = Tokenizer( oov_token = "<OOV>", filters="", lower=False)

In [49]:
tokenizer.fit_on_texts( df['pergunta_limpa'] + " " + df['resposta_limpa'])

In [74]:
vocab_size = len(tokenizer.word_index)
vocab_size

2613

In [139]:
tokenizer.word_index["<START>"]

2

In [51]:
tokenizer.texts_to_sequences(["<START> are you right-handed ? <END>"])

[[2, 17, 6, 2397, 5, 3]]

In [52]:
todos_textos = df['pergunta_limpa'] + " " + df['resposta_limpa']

In [53]:
todos_textos

0       <START> hi how are you doing ? <END> <START> i...
1       <START> i am fine how about yourself ? <END> <...
2       <START> i am pretty good thanks for asking <EN...
3       <START> no problem so how have you been ? <END...
4       <START> i have been great what about you ? <EN...
                              ...                        
3720    <START> that's a good question maybe it is not...
3721    <START> are you right-handed ? <END> <START> y...
3722    <START> yes all my life <END> <START> you are ...
3723    <START> you are wearing out your right hand st...
3724    <START> but i do all my writing with my right ...
Length: 3725, dtype: object

In [96]:
maior_sequencia = 0
for frase in todos_textos:
    tokens = frase.split(" ")
    if len(tokens) > maior_sequencia:
        maior_sequencia = len(tokens)
maior_sequencia

37

In [97]:
perguntas_sequencia = tokenizer.texts_to_sequences(df["pergunta_limpa"])

In [98]:
respostas_sequencia = tokenizer.texts_to_sequences(df["resposta_limpa"])
respostas_sequencia_nostart = [ elemento[1:] for elemento in respostas_sequencia] 

In [99]:
perguntas_padded = pad_sequences(perguntas_sequencia, padding="post", maxlen=maior_sequencia)
print("Perguntas Shape: ", perguntas_padded.shape)
respostas_padded = pad_sequences(respostas_sequencia_nostart, padding="post", maxlen=maior_sequencia)
print("Respostas Shape: ", respostas_padded.shape)

Perguntas Shape:  (3725, 37)
Respostas Shape:  (3725, 37)


In [100]:
respostas_padded

array([[   4,   31,  587, ...,    0,    0,    0],
       [   4,   31,  161, ...,    0,    0,    0],
       [  35,  165,   27, ...,    0,    0,    0],
       ...,
       [   6,   17, 1367, ...,    0,    0,    0],
       [  34,    4,   14, ...,    0,    0,    0],
       [ 250, 1483,  960, ...,    0,    0,    0]])

In [105]:
decoder_output_data = to_categorical(respostas_padded, num_classes=vocab_size + 1)
decoder_input_data = respostas_padded
encoder_input_data = perguntas_padded

In [106]:
print("Decoder Output Shape:", decoder_output_data.shape)
print("Vocab Size:", vocab_size)

Decoder Output Shape: (3725, 37, 2614)
Vocab Size: 2613


In [109]:
encoder_input = Input(shape=(maior_sequencia, ))
encoder_embedding = Embedding(vocab_size + 1, 150, mask_zero=True)(encoder_input)
encoder_outputs, state_h, state_c = LSTM(150, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_input = Input(shape=(maior_sequencia, ))
decoder_embedding = Embedding(vocab_size + 1, 150, mask_zero=True)(decoder_input)
decoder_lstm = LSTM(150, return_state=True, return_sequences=True)

decoder_output, _, _ = decoder_lstm(decoder_embedding, initial_state = encoder_states)
decoder_dense = Dense(vocab_size + 1, activation="softmax")
output = decoder_dense (decoder_output)

rms = RMSprop()

modelo = Model([encoder_input, decoder_input], output)
modelo.compile(optimizer=rms, loss="categorical_crossentropy", metrics=["accuracy"])
modelo.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)   │ (None, 37)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_11 (InputLayer)   │ (None, 37)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_10 (Embedding)      │ (None, 37, 150)           │         392,100 │ input_layer_10[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ not_equal_10 (NotEqual)       │ (None, 37)                │               0 │ input_layer_10[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_11 (Embedding)      │ (None, 37, 150)           │         392,100 │ input_layer_11[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_10 (LSTM)                │ [(None, 150), (None,      │         180,600 │ embedding_10[0][0],        │
│                               │ 150), (None, 150)]        │                 │ not_equal_10[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_11 (LSTM)                │ [(None, 37, 150), (None,  │         180,600 │ embedding_11[0][0],        │
│                               │ 150), (None, 150)]        │                 │ lstm_10[0][1],             │
│                               │                           │                 │ lstm_10[0][2]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 37, 2614)          │         394,714 │ lstm_11[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 1,540,114 (5.88 MB)

 Trainable params: 1,540,114 (5.88 MB)

 Non-trainable params: 0 (0.00 B)

In [110]:
history = modelo.fit([encoder_input_data, decoder_input_data], decoder_output_data, batch_size=32, epochs=100)

Epoch 1/100


D:\usr\anaconda312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_60', 'keras_tensor_66']. Received: the structure of inputs=('*', '*')
  warnings.warn(


117/117 ━━━━━━━━━━━━━━━━━━━━ 28s 160ms/step - accuracy: 0.0282 - loss: 6.7057
Epoch 2/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 18s 155ms/step - accuracy: 0.0356 - loss: 5.0262
Epoch 3/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 18s 155ms/step - accuracy: 0.0563 - loss: 4.4884
Epoch 4/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 20s 173ms/step - accuracy: 0.0814 - loss: 3.9841
Epoch 5/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 18s 157ms/step - accuracy: 0.0928 - loss: 3.5472
Epoch 6/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 18s 156ms/step - accuracy: 0.1062 - loss: 3.2106
Epoch 7/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 18s 153ms/step - accuracy: 0.1167 - loss: 2.9712
Epoch 8/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 17s 146ms/step - accuracy: 0.1271 - loss: 2.7429
Epoch 9/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 17s 147ms/step - accuracy: 0.1358 - loss: 2.5445
Epoch 10/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 16s 140ms/step - accuracy: 0.1432 - loss: 2.3558
Epoch 11/100
117/117 ━━━━━━━━━━━━━━━━━━━━ 17s 144ms/step - accuracy: 0.1476 - loss: 2.1841
Epoch 12/100
117/117

In [ ]:
## Modelo da Conversa

In [112]:
encoder_model = Model( encoder_input, encoder_states )
decoder_state_input_h = Input( shape=(150, ) )
decoder_state_input_c = Input( shape=(150, ) )
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_state_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense( decoder_outputs )

decoder_model = Model([decoder_input] + decoder_state_inputs, [decoder_outputs] + decoder_states)
decoder_model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)   │ (None, 37)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_11 (Embedding)      │ (None, 37, 150)           │         392,100 │ input_layer_11[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_12 (InputLayer)   │ (None, 150)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_13 (InputLayer)   │ (None, 150)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_11 (LSTM)                │ [(None, 37, 150), (None,  │         180,600 │ embedding_11[0][0],        │
│                               │ 150), (None, 150)]        │                 │ input_layer_12[0][0],      │
│                               │                           │                 │ input_layer_13[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 37, 2614)          │         394,714 │ lstm_11[1][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 967,414 (3.69 MB)

 Trainable params: 967,414 (3.69 MB)

 Non-trainable params: 0 (0.00 B)

In [113]:
textos = ["how old are you ?"]

In [114]:
textos_sequence = tokenizer.texts_to_sequences( textos )

In [173]:
textos_padded = pad_sequences( textos_sequence, padding="post", maxlen=maior_sequencia)
print("Textos padded: ", textos_padded)

Textos padded:  [[ 41 129  17   6   5   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0]]


In [166]:
state_values = encoder_model.predict( textos_padded )
state_values

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


[array([[ 0.7849532 , -0.77993596,  0.8672302 ,  0.7723921 ,  0.5113992 ,
          0.8420526 , -0.79338735, -0.11028779,  0.15819143,  0.74755865,
         -0.80887014, -0.7023595 , -0.7978558 , -0.7730283 ,  0.88394374,
         -0.7954734 , -0.76091594,  0.09402741,  0.07860922, -0.2980494 ,
          0.77486414, -0.12477224, -0.76243615,  0.16120282, -0.13264218,
         -0.20013037, -0.7767896 ,  0.78776175, -0.82587266,  0.7263624 ,
         -0.7652562 ,  0.32067248,  0.8141385 , -0.61756337, -0.3109777 ,
          0.82958746,  0.8184098 ,  0.8038691 , -0.2926454 ,  0.8490753 ,
         -0.8458056 , -0.14114027, -0.6743366 , -0.190706  ,  0.18335436,
         -0.42096162,  0.724474  , -0.82721424, -0.2448557 , -0.5828115 ,
         -0.78700894, -0.7945573 ,  0.4396975 , -0.21725936, -0.10707984,
          0.8606872 ,  0.3367273 , -0.19556235, -0.2107387 , -0.81171614,
         -0.8081907 ,  0.81810504, -0.6649142 , -0.81113535, -0.7903413 ,
          0.5719911 ,  0.28884146, -0.

In [172]:
target_sequence = np.zeros((1, 1))
target_sequence[0, 0] = tokenizer.word_index["<START>"]
print("Target Sequence: ", target_sequence)
dec_output, saida_state_h, saida_state_c = decoder_model.predict( [target_sequence] + state_values )
state_values = [saida_state_h, saida_state_c]
print("Dec Output: ", dec_output[0, 0])
numero_palavra = np.argmax(dec_output[0, 0])
palavra = tokenizer.sequences_to_texts( [[ numero_palavra ]] )
palavra

Target Sequence:  [[2.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Dec Output:  [0.00028054 0.00026104 0.00025004 ... 0.00102669 0.00050422 0.00097924]


['showers']

In [135]:
np.argmax(dec_output[0, -1])

899

In [136]:
numero_palavra = np.argmax(dec_output[0, 0])
palavra = tokenizer.sequences_to_texts( [[ 899 ]] )
palavra

['spit']

array([[10461.]])

In [186]:
def gerar_texto( texto_entrada ): 
    textos_sequence = tokenizer.texts_to_sequences( [texto_entrada] )
    textos_padded = pad_sequences( textos_sequence, padding="post", maxlen=maior_sequencia)
    print("Textos padded: ", textos_padded)
    state_values = encoder_model.predict( textos_padded )
    palavra_start = tokenizer.word_index["<START>"]
    palavra_end = tokenizer.word_index["<END>"]
    index_palavra = palavra_start
    texto_saida = ""
    max_palavras = 10
    for i in range(max_palavras):
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = index_palavra
        print("Target Sequence: ", target_sequence)
        dec_output, saida_state_h, saida_state_c = decoder_model.predict( [target_sequence] + state_values )
        state_values = [saida_state_h, saida_state_c]
        index_palavra = np.argmax(dec_output[0, 0])
        print("Numero Palavra: ", index_palavra)
        if index_palavra == palavra_end:
            break
        print("Palavra: ", palavra)
        palavra_saida = tokenizer.sequences_to_texts([[ index_palavra ]])[0]
        texto_saida = texto_saida + " " + palavra_saida
    return texto_saida
        

    

In [187]:
gerar_texto("how old are you ?")

Textos padded:  [[ 41 129  17   6   5   0   0   0   0   0   0   0   0   0   0   0   0]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Target Sequence:  [[2.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Numero Palavra:  1361
Palavra:  ['showers']
Target Sequence:  [[1361.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Numero Palavra:  1361
Palavra:  ['showers']
Target Sequence:  [[1361.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Numero Palavra:  534
Palavra:  ['showers']
Target Sequence:  [[534.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Numero Palavra:  534
Palavra:  ['showers']
Target Sequence:  [[534.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Numero Palavra:  534
Palavra:  ['showers']
Target Sequence:  [[534.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Numero Palavra:  534
Palavra:  ['showers']
Target Sequence:  [[534.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Numero Palavra:  534
Palavra:  ['showers']
Target Sequence:  [[534.]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Numero Palavra:  534
Palavra:  ['showers']
Target S

' hopefully hopefully crime crime crime crime crime crime crime crime'